In [2]:
import sys
import os
import logging

# Setup logging to see the conversion process
logging.basicConfig(level=logging.INFO, format='%(levelname)s: %(message)s')

In [ ]:
# Add the parent directory to path to import our updated module
sys.path.insert(0, os.path.join(os.path.dirname(os.getcwd()), 'src'))

# Import local module
from emotion_clf_pipeline.predict import AzureEndpointPredictor

#### Option 1 - Using Private URL (auto-converts to NGROK)

In [3]:
predictor = AzureEndpointPredictor(
    api_key='72fNK2ogONHXje1XShvxvQ3mZUKZN8kErOTKnZBx1xaEd9apKnoZJQQJ99BFAAAAAAAAAAAAINFRAZML3cS9', 
    endpoint_url='http://194.171.191.227:30526/api/v1/endpoint/deberta-emotion-clf-endpoint/score',  # Private URL
    encoders_dir="../models/encoders"
)

INFO: 🔄 Converting private URL to NGROK: http://194.171.191.227:30526/api/v1/endpoint/deberta-emotion-clf-endpoint/score
INFO: ✅ Converted to server 227 NGROK: https://adsai2.ngrok.dev/port-30526/api/v1/endpoint/deberta-emotion-clf-endpoint/score
INFO: 🌐 Azure endpoint initialized with NGROK: https://adsai2.ngrok.dev/port-30526/api/v1/endpoint/deberta-emotion-clf-endpoint/score


### Option 2 - Using NGROK URL directly

In [4]:
predictor = AzureEndpointPredictor(
    api_key='72fNK2ogONHXje1XShvxvQ3mZUKZN8kErOTKnZBx1xaEd9apKnoZJQQJ99BFAAAAAAAAAAAAINFRAZML3cS9', 
    endpoint_url='http://194.171.191.227:30526/api/v1/endpoint/deberta-emotion-clf-endpoint/score',  # Private URL
    encoders_dir="../models/encoders"
)

INFO: 🔄 Converting private URL to NGROK: http://194.171.191.227:30526/api/v1/endpoint/deberta-emotion-clf-endpoint/score
INFO: ✅ Converted to server 227 NGROK: https://adsai2.ngrok.dev/port-30526/api/v1/endpoint/deberta-emotion-clf-endpoint/score
INFO: 🌐 Azure endpoint initialized with NGROK: https://adsai2.ngrok.dev/port-30526/api/v1/endpoint/deberta-emotion-clf-endpoint/score


In [5]:
result = predictor.predict("i'm really admiring what you do")
result

INFO: 🔄 Attempt 1/5 - Sending request to: https://adsai2.ngrok.dev/port-30526/api/v1/endpoint/deberta-emotion-clf-endpoint/score
INFO: ✅ Prediction successful!


{'emotion': 'happiness', 'sub_emotion': 'admiration', 'intensity': 'moderate'}